# Import

In [1]:
import mlflow
import mlflow.keras
import mlflow.tensorflow

import numpy as np
import tensorflow
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

# Setup

In [2]:
mlflow.set_tracking_uri('http://0.0.0.0:5000')

class MlFlowModelCheckpoint(tensorflow.keras.callbacks.ModelCheckpoint):
    def __init__(self, artifact_name, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.artifact_name = artifact_name

    def _save_model(self, epoch, batch, logs):
        super()._save_model(epoch, batch, logs)
        mlflow.keras.log_model(self.model, self.artifact_name)

model_checkpoint_callback = MlFlowModelCheckpoint(
    artifact_name='best_model',
    filepath='../data/checkpoints/best_model.h5',
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

params = dict(
    batch_size = 128,
    num_classes = 10,
    epochs = 10,
)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = tf.keras.utils.to_categorical(y_train, params['num_classes'])
y_test = tf.keras.utils.to_categorical(y_test, params['num_classes'])

# Run

In [3]:
def make_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(params['num_classes'], activation='softmax'))
    return model

with mlflow.start_run(run_name='mnist_keras_tutorial') as run:
    mlflow.tensorflow.autolog(
        every_n_iter=1
    )
    
    model = make_model()
    model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(
                learning_rate=0.00001,
              ),
              metrics=['accuracy'])
    
    history = model.fit(
        x_train,
        y_train,
        batch_size=params['batch_size'],
        epochs=params['epochs'],
        verbose=1,
        validation_data=(x_test, y_test),
        callbacks=[model_checkpoint_callback],
    )

2022-05-30 16:05:33.076606: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
469/469 [==============================] - 20s 40ms/step - loss: 2.3738 - accuracy: 0.1029 - val_loss: 2.3567 - val_accuracy: 0.0991
Epoch 2/10
469/469 [==============================] - 18s 38ms/step - loss: 2.3691 - accuracy: 0.1035 - val_loss: 2.3520 - val_accuracy: 0.1029
Epoch 3/10
469/469 [==============================] - 16s 34ms/step - loss: 2.3645 - accuracy: 0.1046 - val_loss: 2.3473 - val_accuracy: 0.1056
Epoch 4/10
469/469 [==============================] - 17s 36ms/step - loss: 2.3592 - accuracy: 0.1081 - val_loss: 2.3428 - val_accuracy: 0.1091
Epoch 5/10
469/469 [==============================] - 15s 32ms/step - loss: 2.3558 - accuracy: 0.1103 - val_loss: 2.3382 - val_accuracy: 0.1124
Epoch 6/10
469/469 [==============================] - 15s 33ms/step - loss: 2.3519 - accuracy: 0.1131 - val_loss: 2.3338 - val_accuracy: 0.1165
Epoch 7/10
469/469 [==============================] - 16s 34ms/step - loss: 2.3470 - accuracy: 0.1153 - val_loss: 2.3293 - val_accuracy:

2022/05/30 16:08:23 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under file:data/mlflow-artifacts/0/7f4e6104eec8494cbb9f92f5f73ffa56/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the tracking store. If logging to a mlflow server via REST, consider upgrading the server version to MLflow 1.7.0 or above.


# Inspect

In [4]:
runs = mlflow.search_runs(['0'])
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.loss,metrics.accuracy,metrics.val_accuracy,metrics.val_loss,...,params.use_multiprocessing,params.opt_decay,params.opt_nesterov,params.opt_name,tags.mlflow.runName,tags.mlflow.log-model.history,tags.mlflow.source.git.commit,tags.mlflow.source.type,tags.mlflow.user,tags.mlflow.source.name
0,7f4e6104eec8494cbb9f92f5f73ffa56,0,FINISHED,file:data/mlflow-artifacts/0/7f4e6104eec8494cb...,2022-05-30 07:05:32.734000+00:00,2022-05-30 07:08:23.722000+00:00,2.335804,0.121500,0.1309,2.316342,...,False,0.0,False,SGD,mnist_keras_tutorial,"[{""run_id"": ""7f4e6104eec8494cbb9f92f5f73ffa56""...",70b23638f4afca187b0b568ffb7fa53e30477ff3,LOCAL,shin,/Users/shin/.anyenv/envs/pyenv/versions/3.7.13...
1,120397229f69452b8855da6cbc809e48,0,FINISHED,file:data/mlflow-artifacts/0/120397229f69452b8...,2022-05-30 06:59:17.170000+00:00,2022-05-30 07:00:27.651000+00:00,2.312257,0.132933,0.1670,2.281477,...,False,0.0,False,SGD,mnist_keras_tutorial,"[{""run_id"": ""120397229f69452b8855da6cbc809e48""...",70b23638f4afca187b0b568ffb7fa53e30477ff3,LOCAL,shin,/Users/shin/.anyenv/envs/pyenv/versions/3.7.13...
2,3ed5405677b44432b57a94a9e58ced68,0,FAILED,file:data/mlflow-artifacts/0/3ed5405677b44432b...,2022-05-29 15:14:44.192000+00:00,2022-05-29 15:29:35.840000+00:00,2.215092,0.205950,0.2826,2.190054,...,False,0.0,False,SGD,mnist_keras_tutorial,"[{""run_id"": ""3ed5405677b44432b57a94a9e58ced68""...",70b23638f4afca187b0b568ffb7fa53e30477ff3,LOCAL,shin,/Users/shin/.anyenv/envs/pyenv/versions/3.7.13...
3,11cef92ae3534fb3b13b6f8b223d4ec1,0,FAILED,file:data/mlflow-artifacts/0/11cef92ae3534fb3b...,2022-05-29 15:08:12.638000+00:00,2022-05-29 15:08:21.502000+00:00,NaN,NaN,NaN,NaN,...,False,0.0,False,SGD,mnist_keras_tutorial,None,70b23638f4afca187b0b568ffb7fa53e30477ff3,LOCAL,shin,/Users/shin/.anyenv/envs/pyenv/versions/3.7.13...


In [7]:
import json
from pprint import pprint

pprint(json.loads(runs["tags.mlflow.log-model.history"].iloc[1]))

[{'artifact_path': 'best_model',
  'flavors': {'keras': {'code': None,
                        'data': 'data',
                        'keras_module': 'tensorflow.keras',
                        'keras_version': '2.9.0',
                        'save_format': 'tf'},
              'python_function': {'data': 'data',
                                  'env': 'conda.yaml',
                                  'loader_module': 'mlflow.keras',
                                  'python_version': '3.7.13'}},
  'mlflow_version': '1.26.1',
  'model_uuid': 'ef0ff40dee434995ace7bc18029831f4',
  'run_id': '120397229f69452b8855da6cbc809e48',
  'utc_time_created': '2022-05-30 06:59:26.123664'},
 {'artifact_path': 'best_model',
  'flavors': {'keras': {'code': None,
                        'data': 'data',
                        'keras_module': 'tensorflow.keras',
                        'keras_version': '2.9.0',
                        'save_format': 'tf'},
              'python_function': {'data': 'data